# PyCWT tutorial
> 15/06/2021<br>
> Thomas.Moore@csiro.au<br>
> https://pycwt.readthedocs.io/en/latest/tutorial.html<br>

---

# SETUP

## define cluster now

In [1]:
# define cluster
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,
                       memory="12GB",
                       walltime="1:00:00",
                       job_extra=['--qos="express"'],
                       processes=1
                      )

In [2]:
# spinup cluster
#how many workers do you want?
cluster.scale_up(n=7)

In [3]:
# setup the scheduler in this notebook.
from distributed import Client
client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.141.1.22:41351 Dashboard: http://10.141.1.22:8787/status,Cluster Workers: 7 Cores: 14 Memory: 78.26 GiB


In [1]:
from __future__ import division
import numpy
from matplotlib import pyplot

import pycwt as wavelet
from pycwt.helpers import find

## load the dataset and define some data related parameters. In this case, the first 19 lines of the data file contain meta-data, that we ignore, since we set them manually (i.e. title, units).

In [2]:
file = '/home/moo270/work/observations/nino3sst.txt'
dat = numpy.genfromtxt(file, skip_header=19)

In [3]:
title = 'NINO3 Sea Surface Temperature'
label = 'NINO3 SST'
units = 'degC'
t0 = 1871.0
dt = 0.25  # In years

In [4]:
# We also create a time array in years.
N = dat.size
t = numpy.arange(0, N) * dt + t0

### We write the following code to detrend and normalize the input data by its standard deviation. Sometimes detrending is not necessary and simply removing the mean value is good enough. However, if your dataset has a well defined trend it is strongly advised to perform detrending. Here, we fit a one-degree polynomial function and then subtract it from the original data.

In [23]:
p = numpy.polyfit(t - t0, dat, 1)
dat_notrend = dat - numpy.polyval(p, t - t0)
std = dat_notrend.std()  # Standard deviation
var = std ** 2  # Variance
dat_norm = dat_notrend / std  # Normalized dataset

### The next step is to define some parameters of our wavelet analysis. We select the mother wavelet, in this case the Morlet wavelet with ω0=6.

In [ ]:
mother = wavelet.Morlet(6)
s0 = 2 * dt  # Starting scale, in this case 2 * 0.25 years = 6 months
dj = 1 / 12  # Twelve sub-octaves per octaves
J = 7 / dj  # Seven powers of two with dj sub-octaves
alpha, _, _ = wavelet.ar1(dat)  # Lag-1 autocorrelation for red noise

## Load tools

In [5]:
#Pangeo toolset
import xarray as xr
import dask as dask

#Basics
import math
import numpy as np
import scipy
import datetime
import pandas as pd
from datetime import datetime as dt
import os

#Plotting
#import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import matplotlib.patches as patches
#import proplot as pplot
#import cmocean
#import cmocean.cm as cmo
#import xcmocean
from cartopy import crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import nc_time_axis

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

from copy import copy


import dask.dataframe as dd

#reload local functions
import importlib

#xgcm
#import xgcm

#xesmf
#import xesmf as xesmf

# turn off warnings
import warnings
warnings.filterwarnings("ignore")

#rechunker
from rechunker import rechunk

%matplotlib inline

#local path imports
import xcache as xc


In [6]:
#local functions
import ACCESS_D_preprocess_helper as helper
importlib.reload(helper)

<module 'ACCESS_D_preprocess_helper' from '/OSM/CBR/OA_DCFP/work/moo270/code/ACCESS_D_preprocess/notebooks/ACCESS_D_preprocess_helper.py'>